In [1]:
import numpy as np
from pyscf import gto, scf, fci


$$
\begin{split}
E_H[\gamma_1] & = \frac{1}{2} \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{a,\kappa} c_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma_1] & = \frac{1}{2} \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_a(r) \phi_b(r')^{\ast}  \phi_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & = \frac{1}{2} \iint \frac{\gamma_1(r,r) \gamma_1(r',r')}{|r-r'|} dr dr' \\
\end{split}
$$

In [2]:
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu,nu,kappa,lamda]

    return 1/2.*energy



$$
\begin{split}
E_x[\gamma_1] & = -\frac{1}{4} \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x[\gamma_1] & = -\frac{1}{4} \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_x[\gamma_1] & = -\frac{1}{4} \iint \frac{\gamma_1(r,r') \gamma_1(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [3]:
def ONERDMFT_exchange_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,b]*C[lamda,a]*Fouridx[mu,nu,kappa,lamda]

    return -1/4.*energy


$$
\begin{split}
E_{xc}[\gamma_1] & = -\frac{1}{4} \sum_a^M \sum_b^M \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_{xc}[\gamma_1] & = -\frac{1}{4} \iint \sum_a^M \sum_b^M \sqrt{n_a n_b} \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_{xc}[\gamma_1] & = -\frac{1}{4} \iint \frac{\gamma_1^{\frac{1}{2}}(r,r') \gamma_1^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [4]:
def ONERDMFT_Mueller_functional(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return -1/4.*energy

In [5]:
mol = gto.Mole()
mol.atom = """
    C    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
C = mf.mo_coeff


N = mol.nelec[0]
gamma=np.matmul(C[:,0:N],C[:,0:N].T)*2

occu, naturalC = np.linalg.eigh(gamma)

converged SCF energy = -37.5882040183502


In [6]:
# run FCI 
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
print(e, mf.e_tot, e-mf.e_tot)

-37.716264429240276 -37.58820401835024 -0.12806041089003628


In [7]:
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)

In [8]:
for i, n  in enumerate(occu):
    if abs(n) < 1e-10:
        occu[i] = 0 

In [9]:
h = mf.get_hcore()
print(np.trace(np.matmul(h,gamma))+ONERDMFT_hartree_energy(eri, naturalC, occu, mol)+ONERDMFT_exchange_energy(eri, naturalC, occu, mol))
print(np.trace(np.matmul(h,FCIgamma))+ONERDMFT_hartree_energy(eri, FCInaturalC, FCIoccu, mol)+ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))

-37.58820401835027
-39.069427893700535


In [10]:
print("Mueller Energy, Exchange Energy of HF-gamma:", ONERDMFT_Mueller_functional(eri, naturalC, occu, mol),ONERDMFT_exchange_energy(eri, naturalC, occu, mol))
#print("Mueller Correlation Energy:", ONERDMFT_Mueller_functional(eri, FCInaturalC, FCIoccu, mol)-ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))
print("Mueller Energy, Exchange Energy of FCI-gamma:", ONERDMFT_Mueller_functional(eri, FCInaturalC, FCIoccu, mol),ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))


Mueller Energy, Exchange Energy of HF-gamma: -2.9895518039204294 -5.029550758276508
Mueller Energy, Exchange Energy of FCI-gamma: -3.1510324268759384 -5.43206095070751
